## Fast Agent
Build a SQL Agent fast

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai:gpt-5")

In [4]:
from langchain_core.tools import tool
    
@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    #query = _safe_sql(query)
    q = query
    if q.startswith("Error:"):
        return q
    try:
        return db.run(q)
    except Exception as e:
        return f"Error: {e}"


In [5]:
SYSTEM = f"""You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [6]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

#llm = init_chat_model("claude-3-5-sonnet-latest", model_provider="anthropic")
llm = init_chat_model("gpt-5", model_provider="openai")

agent = create_agent(
    model=llm,
    tools=[execute_sql],
    prompt=SystemMessage(content=SYSTEM),
)

In [7]:
question = "Which table has the largest number of entries?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which table has the largest number of entries?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_o8OQQc9uJ7E46S9Qnjg4Ykqe)
 Call ID: call_o8OQQc9uJ7E46S9Qnjg4Ykqe
  Args:
    query: SELECT m.name AS table_name, SUM(s.ncell) AS approx_rows
FROM sqlite_master AS m
JOIN dbstat AS s
  ON s.name = m.name
WHERE m.type = 'table'
  AND m.name NOT LIKE 'sqlite_%'
  AND s.pagetype = 'leaf'
GROUP BY m.name
ORDER BY approx_rows DESC
LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

[('PlaylistTrack', 8715)]
================================== Ai Message ==================================

PlaylistTrack — approximately 8,715 rows.


In [8]:
question = "Which genre on average has the longest tracks?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which genre on average has the longest tracks?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_Lko2hdjFh27C0yqFdLxANU5D)
 Call ID: call_Lko2hdjFh27C0yqFdLxANU5D
  Args:
    query: SELECT g.Name AS genre, AVG(t.Milliseconds) AS avg_milliseconds
FROM tracks t
JOIN genres g ON g.GenreId = t.GenreId
GROUP BY g.GenreId, g.Name
ORDER BY avg_milliseconds DESC
LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: tracks
[SQL: SELECT g.Name AS genre, AVG(t.Milliseconds) AS avg_milliseconds
FROM tracks t
JOIN genres g ON g.GenreId = t.GenreId
GROUP BY g.GenreId, g.Name
ORDER BY avg_milliseconds DESC
LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================


In [9]:
question = "What are all the tables?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What are all the tables?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_9cnXpL6CDO6cfbf1A9v638GV)
 Call ID: call_9cnXpL6CDO6cfbf1A9v638GV
  Args:
    query: SELECT name FROM sqlite_master WHERE type = 'table' AND name NOT LIKE 'sqlite_%' ORDER BY name;
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================

The database has these tables:
- Album
- Artist
- Customer
- Employee
- Genre
- Invoice
- InvoiceLine
- MediaType
- Playlist
- PlaylistTrack
- Track


Let's try this in Studio